# Introdução

Dado o cenário de ano eleitoral no Brasil, este trabalho procura entender de maneira simplificada o contexto atual das eleições. Utilizaremos dados obtidos no Twitter, rede social em que os candidatos e candidatas geralmente propagam mensagens com suas ideias e propostas. Lá também é possível encontrar a manifestação dos sentimentos dos eleitores através de suas opiniões sobre os candidatos, sejam elas contra ou a favor.

Os dados foram coletados diretamente da API do Twitter, e inclui 100 tweets recentes de cada um dos três principais candidatos - de acordo com as pesquisas eleitorais -, assim como diversas respostas (replies) para cada um dos tweets de cada candidato.

Procuramos responder alguns questionamentos:
- Quais os principais tópicos que estão sendo abordados pelos candidatos?
- Quão similar é o discurso entre os candidatos?
- Qual o sentimento dos eleitores sobre cada candidato?

Com as repostas a estas perguntas, acreditamos que seja possível visualizar um pouco melhor o cenário das eleições para presidência do Brasil, que em breve serão iniciadas.

Além disso, é importante mencionar que os resultados apresentados por este notebook podem variar entre diferentes execuções. Do mesmo modo, os exemplos apresentados no vídeo do projeto podem ser distintos dos que aparecem neste notebook e em suas futuras execuções.

##Imports

In [1]:
import re
import pandas as pd
import json
import nltk
import numpy as np

#Imports similaridade
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Extração de Dados

In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [14]:
def normaliza(df, coluna_alvo='text'):
  coluna_normal = coluna_alvo+"_normal"
  df[coluna_normal] = df[coluna_alvo].str.lower()
  df[coluna_normal].replace(to_replace = '[^a-zA-ZÀ-ÖØ-öø-ÿ\s]', value='', regex = True, inplace = True)
  return df
  
def tokeniza(df, coluna_alvo='text_normal'):
  stop_words = nltk.corpus.stopwords.words('portuguese')
  todos_tokens = []
  todos_textos_tokens = []

  for texto_normal in df[coluna_alvo].to_list():
    tokens = nltk.word_tokenize(texto_normal, language="portuguese")
    tokens_filtrados = [token for token in tokens if token not in stop_words]
    todos_tokens.append(tokens_filtrados)
    texto_tokens = ' '.join(tokens_filtrados)
    todos_textos_tokens.append(texto_tokens)


  df_tokens = pd.DataFrame({'tokens': todos_tokens, 'texto_tokens': todos_textos_tokens})
  df_concat = pd.concat([df, df_tokens], axis="columns")

  return df_concat


def normaliza_tokeniza(df, coluna_alvo = 'text'):
  df_tokens = tokeniza(normaliza(df, coluna_alvo), coluna_alvo)
  return df_tokens

  #word_regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+" #para capturarmos palavras dentro do tweet
  #df['text'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['text'], flags=re.MULTILINE)  #para remover os links do campo de texto do tweet

# Modelagem de Tópicos

*texto em itálico*# Análise de Sentimentos

# Similaridade de Textos

In [16]:
def monta_docs_tweets(df, coluna_alvo, candidatos):
    lista_tweets = []
    for cand in candidatos:
        df_cand = df[df['candidato'] == cand]
        lista_tweets_cand = df_cand[coluna_alvo].to_list()
        lista_tweets.append(lista_tweets_cand)
    
    docs = np.concatenate(lista_tweets)
    return docs

def monta_docs_concat(df, topico, coluna_alvo, candidatos):
    docs = []
    for cand in candidatos:
        df_filtrado = df[df['topico'] == topico and df['candidato'] == cand]
        texto = ' '.join(df_filtrado[coluna_alvo].to_list())
        docs.append(texto)
 
    return docs

def vetoriza_docs(lista_docs):
    corpus_vetorizado = np.vectorize(lista_docs)
    return corpus_vetorizado

def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

def calcula_similaridade(df, topico, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat(df, topico, coluna_alvo, candidatos)
    corpus_vetorizado = vetoriza_docs(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_vetorizado)
    df_sim = similaridade_pares(matriz_tfidf)

    return df_sim